In [1]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile

from collections import defaultdict
from io import StringIO
import matplotlib.pyplot as plt
from PIL import Image

In [2]:
# This is needed to display the images.
%matplotlib inline

# This is needed since the notebook is stored in the object_detection folder.
sys.path.append("..")

In [3]:
from utils import label_map_util

from utils import visualization_utils as vis_util

In [4]:
# What model to download.
MODEL_NAME = 'ssd_mobilenet_v1_coco_2017_11_17'
MODEL_FILE = MODEL_NAME + '.tar.gz'
DOWNLOAD_BASE = 'http://download.tensorflow.org/models/object_detection/'

# Path to frozen detection graph. This is the actual model that is used for the object detection.
PATH_TO_CKPT = MODEL_NAME + '/frozen_inference_graph.pb'

# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = os.path.join('data', 'mscoco_label_map.pbtxt')

NUM_CLASSES = 90

In [5]:
opener = urllib.request.URLopener()
opener.retrieve(DOWNLOAD_BASE + MODEL_FILE, MODEL_FILE)
tar_file = tarfile.open(MODEL_FILE)
for file in tar_file.getmembers():
  file_name = os.path.basename(file.name)
  if 'frozen_inference_graph.pb' in file_name:
    tar_file.extract(file, os.getcwd())

In [6]:
detection_graph = tf.Graph()
with detection_graph.as_default():
  od_graph_def = tf.GraphDef()
  with tf.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
    serialized_graph = fid.read()
    od_graph_def.ParseFromString(serialized_graph)
    tf.import_graph_def(od_graph_def, name='')

In [7]:
label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = label_map_util.create_category_index(categories)

In [9]:
from pykinect2 import PyKinectV2
from pykinect2.PyKinectV2 import *
from pykinect2 import PyKinectRuntime
from acquisitionKinect import AcquisitionKinect
from frame import Frame as Frame
import numpy as np
import cv2
import ctypes
import _ctypes
import sys
import face_recognition
import os

pykinect = PyKinectRuntime.PyKinectRuntime(PyKinectV2.FrameSourceTypes_Depth)
kinect = AcquisitionKinect()
f = Frame()

known_face_encodings = []
known_face_names = []

# Load sample pictures and learn how to recognize it.
dirname = 'face_recognition/knowns'
files = os.listdir(dirname)
for filename in files:
    name, ext = os.path.splitext(filename)
    if ext == '.jpg':
        known_face_names.append(name)
        pathname = os.path.join(dirname, filename)
        img = face_recognition.load_image_file(pathname)
        face_encoding = face_recognition.face_encodings(img)[0]
        known_face_encodings.append(face_encoding)

# Initialize some variables
face_locations = []
face_encodings = []
face_names = []
process_this_frame = True

with detection_graph.as_default():
  with tf.Session(graph=detection_graph) as sess:
    while True:
        
        # --- Getting frames and drawing
        kinect.get_frame(f)
        kinect.get_color_frame()
        image_np = kinect._frameRGB
        image_depth = kinect._frameDepthQuantized 
        image_np = cv2.cvtColor(image_np, cv2.COLOR_RGBA2RGB)
        cv2.imshow("depth", image_depth)

        # ----- Face Recognition
        # Resize frame of video to 1/4 size for faster face recognition processing
        rgb_small_frame = cv2.resize(image_np, (0, 0), fx=0.25, fy=0.25)

        # Only process every other frame of video to save time
        if process_this_frame:
            # Find all the faces and face encodings in the current frame of video
            face_locations = face_recognition.face_locations(rgb_small_frame)
            face_encodings = face_recognition.face_encodings(rgb_small_frame, face_locations)

            face_names = []
            for face_encoding in face_encodings:
                # See if the face is a match for the known face(s)
                distances = face_recognition.face_distance(known_face_encodings, face_encoding)
                min_value = min(distances)

                # tolerance: How much distance between faces to consider it a match. Lower is more strict.
                # 0.6 is typical best performance.
                name = "Unknown"
                if min_value < 0.9:
                    index = np.argmin(distances)
                    name = known_face_names[index]

                face_names.append(name)

        process_this_frame = not process_this_frame

        # Display the results
        for (top, right, bottom, left), name in zip(face_locations, face_names):
            # Scale back up face locations since the frame we detected in was scaled to 1/4 size
            top *= 4
            right *= 4
            bottom *= 4
            left *= 4

            # Draw a box around the face
            cv2.rectangle(image_np, (left, top), (right, bottom), (0, 0, 255), 2)
            # Draw a label with a name below the face
            cv2.rectangle(image_np, (left, bottom - 35), (right, bottom), (0, 0, 255), cv2.FILLED)
            font = cv2.FONT_HERSHEY_DUPLEX
            cv2.putText(image_np, name, (left + 6, bottom - 6), font, 1.0, (255, 255, 255), 1)


        # ----- Object Recognition
        # Definite input and output Tensors for detection_graph
        image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
        # Each box represents a part of the image where a particular object was detected.
        detection_boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
        # Each score represent how level of confidence for each of the objects.
        # Score is shown on the result image, together with the class label.
        detection_scores = detection_graph.get_tensor_by_name('detection_scores:0')
        detection_classes = detection_graph.get_tensor_by_name('detection_classes:0')
        num_detections = detection_graph.get_tensor_by_name('num_detections:0')

        # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
        image_np_expanded = np.expand_dims(image_np, axis=0)
        # Actual detection.
        (boxes, scores, classes, num) = sess.run(
            [detection_boxes, detection_scores, detection_classes, num_detections],
            feed_dict={image_tensor: image_np_expanded})
        # Visualization of the results of a detection.
        vis_util.visualize_boxes_and_labels_on_image_array(
            image_np,
            np.squeeze(boxes),
            np.squeeze(classes).astype(np.int32),
            np.squeeze(scores),
            category_index,
            use_normalized_coordinates=True,
            line_thickness=8)

        # Get coordinates of detected boxes ; ymin, ymax, xmin, xmax
        coordinates = vis_util.return_coordinates(
                    image_np,
                    np.squeeze(boxes),
                    np.squeeze(classes).astype(np.int32),
                    np.squeeze(scores),
                    category_index,
                    use_normalized_coordinates=True,
                    line_thickness=8,
                    min_score_thresh=0.6)
        if coordinates is not None:

            # ----- Face Recognition
            print(face_names)
            count = 0
            for i in range(len(coordinates)):
                if coordinates[i][0] == 'person':
                    if count < len(face_names):
                        coordinates[i][0] = face_names[count]
                        count += 1

            #print(*coordinates)
            print(coordinates)
            xpoint_for_depth=[]
            ypoint_for_depth=[]

            print("object 개수:", len(coordinates))

            # Connect Color and Depth Info.
            frame = pykinect.get_last_depth_frame()
            #frameD = pykinect._depth_frame_data
            frameD = kinect.get_last_depth_frame()
            frame = frame.astype(np.uint8)
            frame = np.reshape(frame, (424, 512))
            frame = cv2.cvtColor(frame, cv2.COLOR_GRAY2RGB)
            cv2.imshow("depth_frame", frame)

            # Take just one point : middle
            
            for i in range(len(coordinates)):

                ypoint_for_depth.append(int((coordinates[i][1]+coordinates[i][2])/2))
                xpoint_for_depth.append(int((coordinates[i][3]+coordinates[i][4])/2))

            print("ypoint_for_depth:", ypoint_for_depth)
            print("xpoint_for_depth:", xpoint_for_depth)

            ##output = cv2.bilateralFilter(output, 1, 150, 75)
            #cv2.imshow('KINECT Video Stream', frame)

            for i in range(len(coordinates)):
                Pixel_Depth = frameD[((ypoint_for_depth[i] * 512) + xpoint_for_depth[i])]
                print("점 하나만 고려:",i+1, "번째 object의 depth value: ", Pixel_Depth)

            # Get average depth value considering all coordinate pairs. (List Version)
            '''Pixel_Depth_List = []
            Pixel_Depth_Sum = 0
            non_error_count = 0
            for i in range(len(coordinates)):
                for x in range(int(coordinates[i][3]), int(coordinates[i][4]+1)):
                    for y in range(int(coordinates[i][1]), int(coordinates[i][2]+1)):
                        Pixel_Depth_Sum += frameD[((y * 512) + x)]
                        if frameD[((y * 512) + x)] != 0:
                            non_error_count += 1

                Pixel_Depth_List.append(int(Pixel_Depth_Sum/non_error_count))

            for i in range(len(coordinates)):
                print("전체 면적 고려:", i+1, "번째 object의 depth value: ", Pixel_Depth_List[i])
            print("\n")'''

            # Get average depth value considering all coordinate pairs. (Dict Version)
            '''Pixel_Depth_Dict = {}
            Pixel_Depth_Sum = 0
            non_error_count = 0
            for i in range(len(coordinates)):

                # ----- Depth Recognition
                for x in range(int(coordinates[i][3]), int(coordinates[i][4]+1)):
                    for y in range(int(coordinates[i][1]), int(coordinates[i][2]+1)):
                        Pixel_Depth_Sum += frameD[((y * 512) + x)]
                        if frameD[((y * 512) + x)] != 0:
                            non_error_count += 1

                Pixel_Depth_Dict[coordinates[i][0]] = int(Pixel_Depth_Sum/non_error_count)

            # Compare each objects' depth. (Sorting: close -> far)
            sorted(Pixel_Depth_Dict.items(), key=lambda t : t[1])
            print(Pixel_Depth_Dict)
            print("\n")'''

            # VideoFileOutput.write(image_np)
            cv2.imshow('KINECT Color', image_np)
            cv2.waitKey(1)
            
        Skeleton_img = kinect._frameSkeleton

        
        if Skeleton_img is not None:

            cv2.resize(Skeleton_img, (640, 480))
            cv2.imshow('body', Skeleton_img)


        key = cv2.waitKey(1)
        if key == 27: break

['SoraRyu']
[['SoraRyu', 124, 1036, 916, 1295, 74.99417066574097]]
object 개수: 1
ypoint_for_depth: [580]
xpoint_for_depth: [1105]
점 하나만 고려: 1 번째 object의 depth value:  16277
['SoraRyu']
[['SoraRyu', 150, 991, 934, 1288, 81.05663657188416]]
object 개수: 1
ypoint_for_depth: [570]
xpoint_for_depth: [1111]
점 하나만 고려: 1 번째 object의 depth value:  16155
['SoraRyu']
[['SoraRyu', 146, 999, 930, 1291, 80.4694652557373]]
object 개수: 1
ypoint_for_depth: [572]
xpoint_for_depth: [1110]
점 하나만 고려: 1 번째 object의 depth value:  19438
['SoraRyu']
[['SoraRyu', 151, 1007, 934, 1288, 78.35190892219543]]
object 개수: 1
ypoint_for_depth: [579]
xpoint_for_depth: [1111]
점 하나만 고려: 1 번째 object의 depth value:  16126
['SoraRyu']
[['SoraRyu', 155, 1002, 930, 1292, 81.81125521659851]]
object 개수: 1
ypoint_for_depth: [578]
xpoint_for_depth: [1111]
점 하나만 고려: 1 번째 object의 depth value:  0
['SoraRyu']
[['SoraRyu', 153, 1008, 931, 1290, 84.64205265045166]]
object 개수: 1
ypoint_for_depth: [580]
xpoint_for_depth: [1110]
점 하나만 고려: 1 번째 obj

[]
[['person', 9, 971, 939, 1276, 68.98245215415955]]
object 개수: 1
ypoint_for_depth: [490]
xpoint_for_depth: [1107]
점 하나만 고려: 1 번째 object의 depth value:  48892
['SoraRyu']
[['SoraRyu', 152, 1032, 936, 1275, 73.41312170028687]]
object 개수: 1
ypoint_for_depth: [592]
xpoint_for_depth: [1105]
점 하나만 고려: 1 번째 object의 depth value:  16250
['SoraRyu']
[['SoraRyu', 161, 1027, 938, 1273, 72.8176474571228]]
object 개수: 1
ypoint_for_depth: [594]
xpoint_for_depth: [1105]
점 하나만 고려: 1 번째 object의 depth value:  0
['SoraRyu']
[['SoraRyu', 157, 1027, 942, 1271, 74.00745749473572]]
object 개수: 1
ypoint_for_depth: [592]
xpoint_for_depth: [1106]
점 하나만 고려: 1 번째 object의 depth value:  16176
['SoraRyu']
[['SoraRyu', 146, 1031, 939, 1271, 72.72924184799194]]
object 개수: 1
ypoint_for_depth: [588]
xpoint_for_depth: [1105]
점 하나만 고려: 1 번째 object의 depth value:  0
['SoraRyu']
[['SoraRyu', 145, 1040, 938, 1272, 71.65337800979614]]
object 개수: 1
ypoint_for_depth: [592]
xpoint_for_depth: [1105]
점 하나만 고려: 1 번째 object의 depth valu

[]
[['refrigerator', 444, 969, 1175, 1498, 89.31050300598145]]
object 개수: 1
ypoint_for_depth: [706]
xpoint_for_depth: [1336]
점 하나만 고려: 1 번째 object의 depth value:  0
[]
[['refrigerator', 437, 973, 1178, 1507, 90.00356793403625]]
object 개수: 1
ypoint_for_depth: [705]
xpoint_for_depth: [1342]
점 하나만 고려: 1 번째 object의 depth value:  0
[]
[['refrigerator', 441, 958, 1177, 1489, 90.04523754119873]]
object 개수: 1
ypoint_for_depth: [699]
xpoint_for_depth: [1333]
점 하나만 고려: 1 번째 object의 depth value:  0
[]
[['refrigerator', 444, 953, 1172, 1466, 68.74142289161682]]
object 개수: 1
ypoint_for_depth: [698]
xpoint_for_depth: [1319]
점 하나만 고려: 1 번째 object의 depth value:  0
[]
[['refrigerator', 444, 951, 1179, 1487, 75.33038854598999], ['person', 12, 1071, 1414, 1920, 60.32087802886963]]
object 개수: 2
ypoint_for_depth: [697, 541]
xpoint_for_depth: [1333, 1667]
점 하나만 고려: 1 번째 object의 depth value:  0
점 하나만 고려: 2 번째 object의 depth value:  48701
[]
[['refrigerator', 436, 955, 1178, 1473, 80.01452684402466]]
object 개수:

[]
[['person', 6, 1068, 1169, 1917, 67.49565601348877]]
object 개수: 1
ypoint_for_depth: [537]
xpoint_for_depth: [1543]
점 하나만 고려: 1 번째 object의 depth value:  0
[]
[['person', 8, 1067, 1168, 1918, 68.48009824752808]]
object 개수: 1
ypoint_for_depth: [537]
xpoint_for_depth: [1543]
점 하나만 고려: 1 번째 object의 depth value:  0
[]
[['person', 9, 1069, 1175, 1913, 68.08881759643555]]
object 개수: 1
ypoint_for_depth: [539]
xpoint_for_depth: [1544]
점 하나만 고려: 1 번째 object의 depth value:  0
[]
[['person', 12, 1069, 1236, 1920, 64.27230834960938]]
object 개수: 1
ypoint_for_depth: [540]
xpoint_for_depth: [1578]
점 하나만 고려: 1 번째 object의 depth value:  0
[]
[['person', 0, 1073, 1133, 1918, 79.62576150894165]]
object 개수: 1
ypoint_for_depth: [536]
xpoint_for_depth: [1525]
점 하나만 고려: 1 번째 object의 depth value:  0
[]
[['person', 0, 1077, 830, 1906, 84.23804044723511]]
object 개수: 1
ypoint_for_depth: [538]
xpoint_for_depth: [1368]
점 하나만 고려: 1 번째 object의 depth value:  0
[]
[['person', 0, 1080, 588, 1901, 63.335442543029785]]
ob

[]
[]
object 개수: 0
ypoint_for_depth: []
xpoint_for_depth: []
[]
[['dog', 661, 1073, 1011, 1569, 68.40478181838989]]
object 개수: 1
ypoint_for_depth: [867]
xpoint_for_depth: [1290]
점 하나만 고려: 1 번째 object의 depth value:  309
[]
[]
object 개수: 0
ypoint_for_depth: []
xpoint_for_depth: []
[]
[]
object 개수: 0
ypoint_for_depth: []
xpoint_for_depth: []
[]
[]
object 개수: 0
ypoint_for_depth: []
xpoint_for_depth: []
[]
[['person', 292, 1042, 969, 1287, 85.33163070678711]]
object 개수: 1
ypoint_for_depth: [667]
xpoint_for_depth: [1128]
점 하나만 고려: 1 번째 object의 depth value:  0
[]
[['person', 72, 1042, 935, 1245, 72.08999395370483]]
object 개수: 1
ypoint_for_depth: [557]
xpoint_for_depth: [1090]
점 하나만 고려: 1 번째 object의 depth value:  21555
[]
[['person', 80, 1058, 919, 1264, 69.06432509422302]]
object 개수: 1
ypoint_for_depth: [569]
xpoint_for_depth: [1091]
점 하나만 고려: 1 번째 object의 depth value:  0
[]
[['person', 56, 1001, 1032, 1338, 75.8230209350586]]
object 개수: 1
ypoint_for_depth: [528]
xpoint_for_depth: [1185]
점 하나

...

[]
[['person', 405, 1072, 1084, 1909, 76.88316106796265]]
object 개수: 1
ypoint_for_depth: [738]
xpoint_for_depth: [1496]
점 하나만 고려: 1 번째 object의 depth value:  32
[]
[['person', 405, 1076, 1065, 1920, 72.5710391998291]]
object 개수: 1
ypoint_for_depth: [740]
xpoint_for_depth: [1492]
점 하나만 고려: 1 번째 object의 depth value:  101
[]
[['person', 402, 1073, 1080, 1911, 71.67199850082397]]
object 개수: 1
ypoint_for_depth: [737]
xpoint_for_depth: [1495]
점 하나만 고려: 1 번째 object의 depth value:  105
[]
[['person', 397, 1068, 1084, 1906, 76.26714706420898]]
object 개수: 1
ypoint_for_depth: [732]
xpoint_for_depth: [1495]
점 하나만 고려: 1 번째 object의 depth value:  16228
[]
[['person', 397, 1070, 1080, 1907, 75.69360733032227]]
object 개수: 1
ypoint_for_depth: [733]
xpoint_for_depth: [1493]
점 하나만 고려: 1 번째 object의 depth value:  16217
[]
[['person', 403, 1075, 1072, 1920, 75.55139660835266]]
object 개수: 1
ypoint_for_depth: [739]
xpoint_for_depth: [1496]
점 하나만 고려: 1 번째 object의 depth value:  82
[]
[['person', 399, 1070, 1083, 1

[]
[['chair', 485, 1039, 1189, 1900, 88.05135488510132], ['person', 513, 1063, 1181, 1906, 65.9130871295929]]
object 개수: 2
ypoint_for_depth: [762, 788]
xpoint_for_depth: [1544, 1543]
점 하나만 고려: 1 번째 object의 depth value:  0
점 하나만 고려: 2 번째 object의 depth value:  0
[]
[['chair', 480, 1027, 1192, 1897, 88.02874088287354], ['person', 511, 1064, 1180, 1904, 67.60861873626709]]
object 개수: 2
ypoint_for_depth: [753, 787]
xpoint_for_depth: [1544, 1542]
점 하나만 고려: 1 번째 object의 depth value:  32
점 하나만 고려: 2 번째 object의 depth value:  31232
[]
[['chair', 490, 1040, 1195, 1896, 88.7539267539978], ['person', 505, 1068, 1183, 1914, 75.14106035232544]]
object 개수: 2
ypoint_for_depth: [765, 786]
xpoint_for_depth: [1545, 1548]
점 하나만 고려: 1 번째 object의 depth value:  9934
점 하나만 고려: 2 번째 object의 depth value:  0
[]
[['chair', 486, 1040, 1193, 1898, 88.70532512664795], ['person', 506, 1067, 1181, 1914, 80.03933429718018]]
object 개수: 2
ypoint_for_depth: [763, 786]
xpoint_for_depth: [1545, 1547]
점 하나만 고려: 1 번째 object의 d

[]
[['person', 471, 1071, 1147, 1920, 74.10966157913208]]
object 개수: 1
ypoint_for_depth: [771]
xpoint_for_depth: [1533]
점 하나만 고려: 1 번째 object의 depth value:  25521
[]
[['person', 461, 1069, 1154, 1916, 71.60600423812866]]
object 개수: 1
ypoint_for_depth: [765]
xpoint_for_depth: [1535]
점 하나만 고려: 1 번째 object의 depth value:  0
[]
[['person', 457, 1067, 1136, 1919, 73.69976043701172], ['chair', 454, 1061, 1142, 1910, 60.21977663040161]]
object 개수: 2
ypoint_for_depth: [762, 757]
xpoint_for_depth: [1527, 1526]
점 하나만 고려: 1 번째 object의 depth value:  0
점 하나만 고려: 2 번째 object의 depth value:  0
[]
[['person', 462, 1067, 1151, 1920, 75.17509460449219]]
object 개수: 1
ypoint_for_depth: [764]
xpoint_for_depth: [1535]
점 하나만 고려: 1 번째 object의 depth value:  0
[]
[['person', 463, 1066, 1150, 1915, 70.36780118942261]]
object 개수: 1
ypoint_for_depth: [764]
xpoint_for_depth: [1532]
점 하나만 고려: 1 번째 object의 depth value:  15
[]
[['person', 462, 1068, 1164, 1917, 76.26311779022217]]
object 개수: 1
ypoint_for_depth: [765]
xp

[]
[]
object 개수: 0
ypoint_for_depth: []
xpoint_for_depth: []
[]
[]
object 개수: 0
ypoint_for_depth: []
xpoint_for_depth: []
[]
[['chair', 502, 1059, 1180, 1677, 63.54959011077881], ['person', 496, 1071, 1581, 1917, 62.544381618499756]]
object 개수: 2
ypoint_for_depth: [780, 783]
xpoint_for_depth: [1428, 1749]
점 하나만 고려: 1 번째 object의 depth value:  0
점 하나만 고려: 2 번째 object의 depth value:  26190
[]
[['chair', 510, 1075, 1204, 1755, 70.01837491989136]]
object 개수: 1
ypoint_for_depth: [792]
xpoint_for_depth: [1479]
점 하나만 고려: 1 번째 object의 depth value:  0
[]
[['chair', 525, 1071, 1187, 1746, 64.71945643424988]]
object 개수: 1
ypoint_for_depth: [798]
xpoint_for_depth: [1466]
점 하나만 고려: 1 번째 object의 depth value:  0
[]
[['chair', 544, 1061, 1206, 1868, 77.25417613983154]]
object 개수: 1
ypoint_for_depth: [802]
xpoint_for_depth: [1537]
점 하나만 고려: 1 번째 object의 depth value:  0
[]
[['chair', 510, 1066, 1211, 1830, 80.79304695129395]]
object 개수: 1
ypoint_for_depth: [788]
xpoint_for_depth: [1520]
점 하나만 고려: 1 번째 obj

KeyboardInterrupt: 